In [19]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Stock the data better than doing a request every try # Done

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped # Done


### PROBLEME : Incohérence des noms de catégories sur plusieurs année : best motion picture -> best picture
# Special effect -> Visual effects etc

### Les informations récoltées ne sont pas les mêmes ex : foreign film c'est le pays qui est récolté, comment faire ?

In [17]:
### OOP   ### DONE

import time
import requests
from scrapy import Selector
from tqdm import tqdm
import pandas as pd

class Oscar_Scraper:
    
    list_names = []
    list_year = []
    list_category = []
    list_films = []
    list_results = []
    
    def __init__(self, from_time, to_time):
        self.links = {
            'pages':[]
        }
        
        self.categories_interest = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role',
                  'Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        self.categories_individuals = ['Directing', 'Actor', 'Actor in a Leading Role', 'Actress', 'Actress in a Leading Role']
        self.categories_films = ['Outstanding Picture','Outstanding Production','Best Motion Picture', 'Best Picture', 'Foreign Language Film', 
                  'Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)', 'Visual Effects']
        
        #lists of different names for same categories
        self.categories_dictionary = {
                    "Actor": ['Actor', 'Actor in a Leading Role'],
                    "Actress": ['Actress', 'Actress in a Leading Role'],
                    "Picture": ['Outstanding Picture', 'Outstanding Production','Best Motion Picture', 'Best Picture'],
                    "VE" : ['Special Effects', 'Special Visual Effects','Special Achievement Award (Visual Effects)','Visual Effects']
                    }
        
        
        self.from_time = from_time
        self.time_period = range(from_time, to_time + 1)
        self.data = {
            'category':[],
            'film':[],
            'name':[],
            'gender':[],
            'result':[],
            'birthdate':[]
        }
        
        

    def getHTML(self):
        
        for i in self.time_period:
            #time.sleep(10) # Crawlerdelay
            url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
            response_oscars = requests.get(url)
            sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
            self.links['pages'].append(sel)

    def getNominees(self, years, i, category, number_nominees):

        for number in range(number_nominees):   
            # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            # FILM
            path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
            nominee_film = ''.join(self.links['pages'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
            
            self.list_year.append(self.from_time + years)         
            self.list_category.append(category)
            self.list_results.append("Nominee")

    def getWinners(self, years, i, category, number_winners):

        if number_winners == 1: # 1 Winner
        # NAME
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)
            
        
            # FIlM
            path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
            nominee_film = ''.join(self.links['pages'][years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)
           
            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Winner")


        if number_winners == 2: # 2 Winners
            # NAME 1st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # NAME 2st Winner 
            path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
            nominee_name = ''.join(self.links['pages'][years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

            if category == "Directing": # For Directing, titles and names are swapped on the website
                self.list_films.append(nominee_name)
            else:
                self.list_names.append(nominee_name)

            self.list_year.append(self.from_time + years)
            self.list_category.append(category)
            self.list_results.append("Tie-winner")

            # FILM
            path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
            path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

            nominee_film = ''.join(self.links['pages'][years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
            nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)

            winner2_film = ''.join(self.links['pages'][years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
            winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

            if category == "Directing":# For Directing, titles and names are swapped on the website
                self.list_names.append(nominee_film)
            else:
                self.list_films.append(nominee_film)        



    def getDATA(self):
            
        for years in tqdm(range(len(self.links['pages']))):

            categories = self.links['pages'][years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
            
            for i in range(len(categories)):
                if categories[i] in self.categories_individuals:

                    # Need the number of winners (might be ties) and nominated          
                    path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
                    number_of_film_nominated = int(''.join(self.links['pages'][years].css(path_nominated).extract()).split("views-row-")[1])


                    path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
                    trial = ''.join(self.links['pages'][years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
                    if len(trial) == 2:
                        number_winners = 1
                    elif len(trial) == 1:
                        number_winners = 2
                    else:
                        print("problem")

                    ## WINNERS                 
                    self.getWinners(years, i, categories[i], number_winners)     
                    ## NOMINEES
                    self.getNominees(years, i, categories[i], number_of_film_nominated )   
                    
                    
        self.data = {
            'year': self.list_year,
            'category': self.list_category,
            'film': self.list_films,
            'name': self.list_names,
            #'gender':[],
            'result':self.list_results
            #,
            #'birthdate':[]
        }            


    def getINDIVIDUALS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        individuals = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_individuals]]
        return(individuals)
    
    def getFILMS(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        films = df.loc(axis = 0)[pd.IndexSlice[:, self.categories_interest[5:]]]
        films.columns = ('film', 'Studio/Creator(s)', 'result')
        return(films)
    
     #creating big dataframe with all data
    def getAllCategories(self):
        df = pd.DataFrame(self.data)
        df = df.set_index(['year','category'])
        return(df)
    
    #in progress
    """ #creating CSV files based on scraped data. Possible naming of the columns for different datasets
    def create_named_CSV(self,dataframe,categories,columns_name):
        if len(column_name) > 2:
          raise Exception('Items exceeds the maximum allowed length of 2')
        category = []
        if type(categories) == 'str':
        category.append(categories)
        for x in self.categories_dictionary:
            for y in range(len(self.categories_dictionary[x])):
                if category == self.categories_dictionary[x][y]:
                    if x not in names:
                        names.append(x)
        for x in names:
            create_empty_CSV(x)
        df_final.columns = columns_name
             films.to_csv(i + '.csv', sep=' ', mode='a')"""
        
    #creating empty csv names after argument of the method   
    def create_empty_CSV(*stringcsv_names):
        for i in stringcsv_names:
            with open(i + '.csv', 'w') as f:
                pass
        

In [18]:
test = Oscar_Scraper(1990,2000)
test.getHTML()
test.getDATA()


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 11.72it/s]


True